# Data Science - ICARO
_________________________________
Clase: 22

Tema: Proyecto integrador - No Supervisado
_________________________________


## Objetivo:
- Aplicar algoritmos de Clustering (KMeans, DBSCAN) para descubrir agrupaciones naturales en el dataset "Wine Quality".
- Utilizar técnicas de Reducción de Dimensionalidad (PCA, t-SNE, UMAP) para visualizar la estructura de los datos y los clusters.
- Interpretar los resultados de los métodos no supervisados y las visualizaciones, extrayendo conclusiones sobre la organización intrínseca de los datos de vino.

---

## Índice

1. [Imports y Configuración Inicial](#imports)
2. [Carga y Preparación de Datos](#carga)
3. [Análisis Exploratorio de Datos (EDA)](#eda)
4. [**Tarea 1: Clustering (Descubrir Grupos de Vino)**](#clustering)
    - 4.1. [Preparación de Datos para Clustering](#prep_clustering)
    - 4.2. [Modelo de Clustering 1: KMeans](#kmeans)
    - 4.3. [Modelo de Clustering 2: DBSCAN](#dbscan)
    - 4.4. [**(A COMPLETAR) Tareas Finales de Clustering y Conclusión**](#conclusion_clustering)
5. [**Tarea 2: Reducción de Dimensionalidad y Visualización**](#reduccion_dim)
    - 5.1. [Preparación de Datos para Reducción de Dimensionalidad](#prep_reduccion)
    - 5.2. [Técnica de Reducción 1: PCA para Visualización](#pca)
    - 5.3. [Técnica de Reducción 2: t-SNE para Visualización](#tsne)
    - 5.4. [Técnica de Reducción 3: UMAP para Visualización](#umap)
    - 5.5. [**(A COMPLETAR) Tareas Finales de Reducción y Conclusión**](#conclusion_reduccion)
6. [**(A COMPLETAR) Conclusiones Finales del Proyecto**](#final)

---


<a name="imports"></a>
# 1. Imports y Configuración Inicial
En esta primera celda, importaremos todas las librerías que necesitaremos para nuestro análisis de modelos no supervisados.

In [ ]:
pip install umap-learn

In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos y Preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap # Asegúrate de tener instalado 'umap-learn': pip install umap-learn

# Métricas de evaluación para clustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 10


<a name="carga"></a>
## 2. Carga y Preparación de Datos
Vamos a cargar los dos datasets (vino tinto y vino blanco), añadiremos una columna para identificar el tipo de vino y luego los uniremos en un único DataFrame.

In [ ]:
# URLs de los datasets
url_red = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# Carga de los dataframes
df_red = pd.read_csv(url_red, sep=';')
df_white = pd.read_csv(url_white, sep=';')

# Creación de la columna 'wine_type' (0 para tinto, 1 para blanco)
df_red['wine_type'] = 0
df_white['wine_type'] = 1

# Unión de los dataframes
df = pd.concat([df_red, df_white], axis=0)

# Reseteamos el índice
df.reset_index(drop=True, inplace=True)

print("Forma del dataset combinado:", df.shape)
print("\nPrimeras 5 filas:")
display(df.head())

<a name="eda"></a>
## 3. Análisis Exploratorio de Datos (EDA)
Antes de modelar, es crucial entender y preparar nuestros datos. Reutilizaremos el EDA básico del Módulo 2.

In [ ]:
print("Información general del DataFrame:")
df.info()

print("\nEstadísticas descriptivas de las variables numéricas:")
display(df.describe())

print("\nValores nulos por columna:")
print(df.isnull().sum())

plt.figure(figsize=(8, 6))
sns.countplot(x='quality', data=df, palette='viridis')
plt.title('Distribución de la Calidad del Vino')
plt.xlabel('Calidad del Vino (0-10)')
plt.ylabel('Cantidad de Vinos')
plt.show()

# Matriz de correlación de las variables fisicoquímicas
plt.figure(figsize=(12, 10))
# Excluimos las columnas 'wine_type' y 'quality' para la matriz de correlación de features
correlation_matrix = df.drop(['wine_type', 'quality'], axis=1).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de las Features Fisicoquímicas')
plt.show()


### (A COMPLETAR) Análisis del EDA
Basado en los resultados anteriores, comenta tus observaciones sobre:
- La distribución de las variables.
- La presencia de valores nulos (si los hay).
- Las correlaciones entre las características. ¿Hay alguna que te llame la atención?

<a name="clustering"></a>
# 4. Tarea 1: Clustering (Descubrir Grupos de Vino)
En esta sección, aplicaremos algoritmos de clustering para identificar agrupaciones naturales dentro de nuestro dataset de vinos, sin utilizar las etiquetas de `wine_type` o `quality`.

<a name="prep_clustering"></a>
### 4.1. Preparación de Datos para Clustering
Para el clustering, solo usaremos las características fisicoquímicas de los vinos, y es **crucial** que estén escaladas. Las etiquetas originales (`wine_type`, `quality`) las guardaremos para una posterior comparación y análisis.

In [ ]:
# Definir X (features) y las etiquetas originales (y_type, y_quality) para comparación
X = df.drop(['wine_type', 'quality'], axis=1)
y_type = df['wine_type']
y_quality = df['quality']

print(f"Dimensiones de X para clustering: {X.shape}")

# Escalar las características (fundamental para algoritmos basados en distancia)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nPrimeras 5 filas de X escalado:")
print(pd.DataFrame(X_scaled, columns=X.columns).head())

<a name="kmeans"></a>
### 4.2. Modelo de Clustering 1: KMeans
KMeans es un algoritmo de clustering que busca dividir $N$ observaciones en $K$ clusters, donde cada observación pertenece al cluster con la media (centroide) más cercana. Es nuestro punto de partida para identificar grupos.

#### TAREAS EN EL NOTEBOOK:
1.  **Paso 1: Determinar K óptimo (Método del Codo).**
    * Iterar `K` desde 1 hasta 15 (o un rango similar).
    * Para cada `K`, entrenar un modelo `KMeans` con `X_scaled` y guardar la inercia.
    * Graficar la inercia vs. `K`.
    * Identificar el `K_elegido` donde la curva forma un "codo" y justificar su elección en un comentario markdown.

In [ ]:
inertia = []
k_range = range(1, 15) # Probamos K desde 1 hasta 14

for k in k_range:
    # n_init='auto' es el valor por defecto para versiones recientes, o 10 para compatibilidad.
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Método del Codo para KMeans')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inercia')
plt.xticks(k_range)
plt.grid(True)
plt.show()

# (A COMPLETAR) Justifica tu elección de K aquí
# Por ejemplo: "Observando el gráfico del método del codo, el punto de inflexión más pronunciado parece estar en K = X. Por lo tanto, elegimos X clusters."
k_elegido = None # Reemplaza None con el valor de K que elegiste


2.  **Paso 2: Ejecutar KMeans.**
    * Inicializar y entrenar `KMeans` con `n_clusters=K_elegido` y `random_state=42`.
    * Obtener las asignaciones de cluster usando `.fit_predict(X_scaled)`. Guardar en la variable `clusters_kmeans`.
    * Mostrar la cuenta de puntos por cluster (usando `pd.Series(clusters_kmeans).value_counts()`).

In [ ]:
if k_elegido is not None:
    kmeans_final = KMeans(n_clusters=k_elegido, random_state=42, n_init=10)
    clusters_kmeans = kmeans_final.fit_predict(X_scaled)

    print(f"Número de puntos por cluster (KMeans con K={k_elegido}):")
    print(pd.Series(clusters_kmeans).value_counts().sort_index())
else:
    print("Por favor, define 'k_elegido' en la celda anterior.")

3.  **Paso 3: Evaluación de KMeans con Silhouette Score.**
    * Calcular y mostrar el `silhouette_score(X_scaled, clusters_kmeans)`.
    * Opcional: Calcular Calinski-Harabasz Score y Davies-Bouldin Score.

In [ ]:
if 'clusters_kmeans' in locals():
    silhouette_avg_kmeans = silhouette_score(X_scaled, clusters_kmeans)
    print(f"Silhouette Score para KMeans (K={k_elegido}): {silhouette_avg_kmeans:.3f}")

    calinski_kmeans = calinski_harabasz_score(X_scaled, clusters_kmeans)
    davies_kmeans = davies_bouldin_score(X_scaled, clusters_kmeans)
    print(f"Calinski-Harabasz Score para KMeans: {calinski_kmeans:.3f}")
    print(f"Davies-Bouldin Score para KMeans: {davies_kmeans:.3f}")
else:
    print("Por favor, ejecuta las celdas anteriores para generar los clusters de KMeans.")

**Completar curvas de cada métrica**

<a name="dbscan"></a>
### 4.3. Modelo de Clustering 2: DBSCAN
DBSCAN (Density-Based Spatial Clustering of Applications with Noise) es un algoritmo basado en densidad que agrupa puntos que están densamente conectados, marcando como ruido los puntos que se encuentran en regiones de baja densidad.

#### TAREAS EN EL NOTEBOOK:
1.  **Paso 1: Determinar `eps` y `min_samples` óptimos.**
    * Establecer un `min_samples` inicial (ej., `2 * numero_de_features` o 5).
    * Calcular las distancias al `min_samples`-ésimo vecino más cercano para cada punto.
    * Graficar estas distancias ordenadas.
    * Identificar el "codo" en el gráfico para estimar un `eps_elegido` y justificarlo en un comentario markdown.

In [ ]:
# Para min_samples, un valor común es 2 * número_de_dimensiones, o simplemente 5.
# Para este dataset de vinos (11 dimensiones), usemos un min_samples inicial de 22 (2*11).
min_samples_dbscan = 22 # Experimenta con otros valores, ej. 5

# Encontrar las distancias al k-ésimo vecino más cercano
neighbors = NearestNeighbors(n_neighbors=min_samples_dbscan)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)

# Ordenar las distancias del k-ésimo vecino
distances = np.sort(distances[:, min_samples_dbscan-1], axis=0) # Indexamos min_samples-1 porque es el k-ésimo vecino

plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.title(f'Distancia al {min_samples_dbscan}-ésimo Vecino Más Cercano (para estimar eps)')
plt.xlabel('Puntos ordenados por distancia')
plt.ylabel(f'Distancia al {min_samples_dbscan}-ésimo Vecino')
plt.grid(True)
plt.show()

# (A COMPLETAR) Justifica tu elección de eps aquí
# Por ejemplo: "Observando el gráfico de distancias, el 'codo' más pronunciado parece estar alrededor de eps = Y. Por lo tanto, elegimos Y."
eps_elegido = None # Reemplaza None con el valor de eps que elegiste


2.  **Paso 2: Ejecutar DBSCAN.**
    * Inicializar y entrenar `DBSCAN` con `eps=eps_elegido` y `min_samples=min_samples_dbscan`.
    * Obtener las asignaciones de cluster usando `.fit_predict(X_scaled)`. Guardar en `clusters_dbscan`.
    * Mostrar la cuenta de puntos por cluster, destacando el cluster `-1` (ruido).

In [ ]:
if eps_elegido is not None:
    dbscan_final = DBSCAN(eps=eps_elegido, min_samples=min_samples_dbscan)
    clusters_dbscan = dbscan_final.fit_predict(X_scaled)

    print(f"Número de puntos por cluster (DBSCAN con eps={eps_elegido}, min_samples={min_samples_dbscan}):")
    print(pd.Series(clusters_dbscan).value_counts().sort_index())
else:
    print("Por favor, define 'eps_elegido' en la celda anterior.")

3.  **Paso 3: Evaluación de DBSCAN con Silhouette Score (excluyendo ruido).**
    * Calcular y mostrar el `silhouette_score` para DBSCAN, **excluyendo los puntos clasificados como ruido** (`clusters_dbscan != -1`).
    * Opcional: Calcular Calinski-Harabasz Score y Davies-Bouldin Score (excluyendo ruido).

In [ ]:
if 'clusters_dbscan' in locals():
    valid_indices = clusters_dbscan != -1
    if np.unique(clusters_dbscan[valid_indices]).size > 1: # Asegurarse de que haya al menos 2 clusters válidos
        silhouette_avg_dbscan = silhouette_score(X_scaled[valid_indices], clusters_dbscan[valid_indices])
        print(f"Silhouette Score para DBSCAN (excluyendo ruido): {silhouette_avg_dbscan:.3f}")

        calinski_dbscan = calinski_harabasz_score(X_scaled[valid_indices], clusters_dbscan[valid_indices])
        davies_dbscan = davies_bouldin_score(X_scaled[valid_indices], clusters_dbscan[valid_indices])
        print(f"Calinski-Harabasz Score para DBSCAN: {calinski_dbscan:.3f}")
        print(f"Davies-Bouldin Score para DBSCAN: {davies_dbscan:.3f}")
    else:
        print("No hay suficientes clusters válidos (más de 1) en DBSCAN para calcular Silhouette Score.")
else:
    print("Por favor, ejecuta las celdas anteriores para generar los clusters de DBSCAN.")

<a name="conclusion_clustering"></a>
### 4.4. (A COMPLETAR) Tareas Finales de Clustering y Conclusión
No basta con aplicar los algoritmos; es crucial entender qué nos dicen los clusters y cuál enfoque es más adecuado para nuestro dataset de vinos.

#### TAREAS EN EL NOTEBOOK:
1.  **Métricas Clave:**
    * Recopilen los `Silhouette Score` de KMeans y DBSCAN. ¿Cuál algoritmo obtuvo un mejor score y qué implica en términos de cohesión y separación de clusters?

In [ ]:
print("\n--- Comparación de Métricas de Clustering ---")
if 'silhouette_avg_kmeans' in locals():
    print(f"KMeans (K={k_elegido}): Silhouette Score = {silhouette_avg_kmeans:.3f}")
if 'silhouette_avg_dbscan' in locals():
    print(f"DBSCAN (eps={eps_elegido}, min_samples={min_samples_dbscan}): Silhouette Score = {silhouette_avg_dbscan:.3f}")

# (A COMPLETAR) Compara y comenta los scores aquí:
# "Basado en el Silhouette Score, [KMeans/DBSCAN] parece generar clusters con mejor separación y cohesión interna debido a..."

2.  **Análisis de Clusters vs. Etiquetas Originales:**
    * Para el clustering que consideren "mejor" o más interesante, analicen la distribución de las etiquetas `y_type` (tipo de vino: tinto/blanco) y `y_quality` (calidad del vino) dentro de cada cluster.
    * Pueden usar `pd.crosstab()` o gráficos de barras/conteo para visualizar esto.
    * ¿Hay algún cluster que sea predominantemente de un tipo de vino o de una calidad particular?

In [ ]:
# (A COMPLETAR) Elige el conjunto de clusters que consideres más interesante (ej. clusters_kmeans o clusters_dbscan)
clusters_para_analisis = clusters_kmeans # O clusters_dbscan

if 'clusters_para_analisis' in locals():
    # Análisis de clusters vs. Tipo de Vino
    print("\n--- Análisis de Clusters vs. Tipo de Vino Original ---")
    crosstab_type = pd.crosstab(clusters_para_analisis, y_type)
    display(crosstab_type)

    # Análisis de clusters vs. Calidad del Vino
    print("\n--- Análisis de Clusters vs. Calidad del Vino Original ---")
    crosstab_quality = pd.crosstab(clusters_para_analisis, y_quality)
    display(crosstab_quality)

    # (A COMPLETAR) Comenta tus observaciones sobre estos cruces:
    # "Observamos que el cluster X tiene una alta proporción de vinos [tintos/blancos]..."
    # "En cuanto a la calidad, el cluster Y parece agrupar vinos de [alta/baja/media] calidad..."
else:
    print("Por favor, asegúrate de que 'clusters_kmeans' o 'clusters_dbscan' estén definidos.")

3.  **Completar la Tabla de Resultados de Clustering:**
    * Llenen la siguiente tabla resumen con los resultados de ambos algoritmos.

| Algoritmo | Parámetros Clave (K / eps, min_samples) | Nº Clusters Encontrados | Nº Outliers (DBSCAN) | Silhouette Score |
|-----------|-----------------------------------------|-------------------------|----------------------|------------------|
| KMeans    | `K = [Tu K]`                            | `[Tu Nº Clusters]`      | N/A                  | `[Tu Score]`     |
| DBSCAN    | `eps = [Tu eps], min_samples = [Tu min_samples]` | `[Tu Nº Clusters]`      | `[Tu Nº Outliers]`   | `[Tu Score]`     |

4.  **Justificar:**
    * Escriban una conclusión clara: ¿Qué algoritmo de clustering (KMeans o DBSCAN) eligen como el más "revelador" o útil para este dataset y por qué?
    * Consideren la calidad de los clusters, la interpretabilidad de los resultados, la capacidad de manejar formas no esféricas y su capacidad para encontrar patrones significativos.

```
(A COMPLETAR) Mi Conclusión sobre el Mejor Algoritmo de Clustering:

Basado en [métricas de evaluación, análisis de clusters vs. etiquetas originales, y observaciones visuales], considero que el algoritmo [KMeans / DBSCAN] es el más adecuado para descubrir la estructura natural de este dataset de vinos porque...

```

<a name="reduccion_dim"></a>
# 5. Tarea 2: Reducción de Dimensionalidad y Visualización
Dado que nuestros datos tienen 11 características, es difícil visualizarlos. Aplicaremos técnicas de reducción de dimensionalidad para proyectar los datos a 2 dimensiones, permitiéndonos ver la estructura de los clusters encontrados.

<a name="prep_reduccion"></a>
### 5.1. Preparación de Datos para Reducción de Dimensionalidad
La entrada para las técnicas de reducción de dimensionalidad serán los datos escalados (`X_scaled`) que preparamos previamente.

In [ ]:
print(f"Los datos de entrada para la reducción de dimensionalidad son X_scaled con forma: {X_scaled.shape}")


<a name="pca"></a>
### 5.2. Técnica de Reducción 1: PCA para Visualización
PCA (Principal Component Analysis) es una técnica de reducción de dimensionalidad lineal que busca las direcciones (componentes principales) en los datos que maximizan la varianza.

#### TAREAS EN EL NOTEBOOK:
1.  **Paso 1: Aplicar PCA.**
    * Inicializar `PCA(n_components=2, random_state=42)`.
    * Entrenar y transformar `X_scaled` usando `.fit_transform()`. Guardar en `X_pca`.

In [ ]:
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"Forma de los datos después de PCA: {X_pca.shape}")

2.  **Paso 2: Visualizar Clusters de KMeans en PCA.**
    * Crear un `scatter plot` de `X_pca` coloreado por `clusters_kmeans`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_kmeans' in locals():
    plt.figure(figsize=(10, 8))
    scatter_kmeans_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_kmeans, cmap='viridis', alpha=0.7)
    plt.title(f'KMeans Clusters (K={k_elegido}) en Proyección PCA')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.colorbar(scatter_kmeans_pca, label='Cluster KMeans', ticks=range(k_elegido))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de KMeans para obtener 'clusters_kmeans'.")

3.  **Paso 3: Visualizar Clusters de DBSCAN en PCA.**
    * Crear un `scatter plot` de `X_pca` coloreado por `clusters_dbscan`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_dbscan' in locals():
    plt.figure(figsize=(10, 8))
    scatter_dbscan_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_dbscan, cmap='viridis', alpha=0.7)
    plt.title(f'DBSCAN Clusters (eps={eps_elegido}, min_samples={min_samples_dbscan}) en Proyección PCA')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.colorbar(scatter_dbscan_pca, label='Cluster DBSCAN')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de DBSCAN para obtener 'clusters_dbscan'.")

4.  **Paso 4: Comparación con Tipo de Vino Original.**
    * Crear un `scatter plot` de `X_pca` coloreado por `y_type` (la etiqueta original de tinto/blanco).
    * Añadir título, etiquetas de ejes y una barra de color interpretativa.
    * Comentar las observaciones sobre la separación.

In [ ]:
plt.figure(figsize=(10, 8))
scatter_original_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_type, cmap='coolwarm', alpha=0.7)
plt.title('Tipo de Vino Original en Proyección PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.colorbar(scatter_original_pca, ticks=[0, 1], format=plt.FuncFormatter(lambda x, p: 'Tinto' if x==0 else 'Blanco'), label='Tipo de Vino')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# (A COMPLETAR) Comenta aquí tus observaciones sobre la separación con PCA
# "PCA logra [una buena/mala] separación visual de los vinos por tipo, lo que indica..."
# "Los clusters encontrados por KMeans/DBSCAN se [alinean/no alinean] bien con la separación de PCA..."

<a name="tsne"></a>
### 5.3. Técnica de Reducción 2: t-SNE para Visualización
t-SNE (t-Distributed Stochastic Neighbor Embedding) es una técnica no lineal que se enfoca en preservar las relaciones de proximidad local, lo que la hace excelente para visualizar agrupamientos (clusters).

#### TAREAS EN EL NOTEBOOK:
1.  **Paso 1: Aplicar t-SNE.**
    * Inicializar `TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)`.
    * Entrenar y transformar `X_scaled` usando `.fit_transform()`. Guardar en `X_tsne`. (¡Este paso puede tardar varios segundos/minutos!).

In [ ]:
print("Ejecutando t-SNE... (esto puede tomar un tiempo considerable)")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

print(f"Forma de los datos después de t-SNE: {X_tsne.shape}")

2.  **Paso 2: Visualizar Clusters de KMeans en t-SNE.**
    * Crear un `scatter plot` de `X_tsne` coloreado por `clusters_kmeans`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_kmeans' in locals():
    plt.figure(figsize=(10, 8))
    scatter_kmeans_tsne = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters_kmeans, cmap='viridis', alpha=0.7)
    plt.title(f'KMeans Clusters (K={k_elegido}) en Proyección t-SNE')
    plt.xlabel('t-SNE Componente 1')
    plt.ylabel('t-SNE C omponente 2')
    plt.colorbar(scatter_kmeans_tsne, label='Cluster KMeans', ticks=range(k_elegido))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de KMeans para obtener 'clusters_kmeans'.")

3.  **Paso 3: Visualizar Clusters de DBSCAN en t-SNE.**
    * Crear un `scatter plot` de `X_tsne` coloreado por `clusters_dbscan`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_dbscan' in locals():
    plt.figure(figsize=(10, 8))
    scatter_dbscan_tsne = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters_dbscan, cmap='viridis', alpha=0.7)
    plt.title(f'DBSCAN Clusters (eps={eps_elegido}, min_samples={min_samples_dbscan}) en Proyección t-SNE')
    plt.xlabel('t-SNE Componente 1')
    plt.ylabel('t-SNE Componente 2')
    plt.colorbar(scatter_dbscan_tsne, label='Cluster DBSCAN')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de DBSCAN para obtener 'clusters_dbscan'.")

4.  **Paso 4: Comparación con Tipo de Vino Original.**
    * Crear un `scatter plot` de `X_tsne` coloreado por `y_type`.
    * Añadir título, etiquetas de ejes y una barra de color interpretativa.
    * Comentar las observaciones sobre la separación y la preservación local.

In [ ]:
plt.figure(figsize=(10, 8))
scatter_original_tsne = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_type, cmap='coolwarm', alpha=0.7)
plt.title('Tipo de Vino Original en Proyección t-SNE')
plt.xlabel('t-SNE Componente 1')
plt.ylabel('t-SNE Componente 2')
plt.colorbar(scatter_original_tsne, ticks=[0, 1], format=plt.FuncFormatter(lambda x, p: 'Tinto' if x==0 else 'Blanco'), label='Tipo de Vino')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# (A COMPLETAR) Comenta aquí tus observaciones sobre la separación con t-SNE
# "t-SNE logra [una mejor/peor] separación visual de los clusters/tipos de vino en comparación con PCA, lo que sugiere..."
# "Observamos que t-SNE tiende a formar agrupaciones más densas para los clusters..."

<a name="umap"></a>
### 5.4. Técnica de Reducción 3: UMAP para Visualización
UMAP (Uniform Manifold Approximation and Projection) es una técnica de reducción de dimensionalidad no lineal más reciente y a menudo más rápida que t-SNE, que busca un balance entre la preservación de la estructura local y global.

#### TAREAS EN EL NOTEBOOK:
1.  **Paso 1: Aplicar UMAP.**
    * Inicializar `umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)` (puedes ajustar `n_neighbors` y `min_dist`).
    * Entrenar y transformar `X_scaled` usando `.fit_transform()`. Guardar en `X_umap`.

In [ ]:
print("Ejecutando UMAP... (generalmente más rápido que t-SNE)")
reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1) # Ajusta parámetros si es necesario
X_umap = reducer.fit_transform(X_scaled)

print(f"Forma de los datos después de UMAP: {X_umap.shape}")

2.  **Paso 2: Visualizar Clusters de KMeans en UMAP.**
    * Crear un `scatter plot` de `X_umap` coloreado por `clusters_kmeans`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_kmeans' in locals():
    plt.figure(figsize=(10, 8))
    scatter_kmeans_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=clusters_kmeans, cmap='viridis', alpha=0.7)
    plt.title(f'KMeans Clusters (K={k_elegido}) en Proyección UMAP')
    plt.xlabel('UMAP Componente 1')
    plt.ylabel('UMAP Componente 2')
    plt.colorbar(scatter_kmeans_umap, label='Cluster KMeans', ticks=range(k_elegido))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de KMeans para obtener 'clusters_kmeans'.")

3.  **Paso 3: Visualizar Clusters de DBSCAN en UMAP.**
    * Crear un `scatter plot` de `X_umap` coloreado por `clusters_dbscan`.
    * Añadir título, etiquetas de ejes y una barra de color.

In [ ]:
if 'clusters_dbscan' in locals():
    plt.figure(figsize=(10, 8))
    scatter_dbscan_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=clusters_dbscan, cmap='viridis', alpha=0.7)
    plt.title(f'DBSCAN Clusters (eps={eps_elegido}, min_samples={min_samples_dbscan}) en Proyección UMAP')
    plt.xlabel('UMAP Componente 1')
    plt.ylabel('UMAP Componente 2')
    plt.colorbar(scatter_dbscan_umap, label='Cluster DBSCAN')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Por favor, ejecuta la sección de DBSCAN para obtener 'clusters_dbscan'.")

4.  **Paso 4: Comparación con Tipo de Vino Original.**
    * Crear un `scatter plot` de `X_umap` coloreado por `y_type`.
    * Añadir título, etiquetas de ejes y una barra de color interpretativa.
    * Comentar las observaciones sobre la separación, velocidad y preservación global/local.

In [ ]:
plt.figure(figsize=(10, 8))
scatter_original_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y_type, cmap='coolwarm', alpha=0.7)
plt.title('Tipo de Vino Original en Proyección UMAP')
plt.xlabel('UMAP Componente 1')
plt.ylabel('UMAP Componente 2')
plt.colorbar(scatter_original_umap, ticks=[0, 1], format=plt.FuncFormatter(lambda x, p: 'Tinto' if x==0 else 'Blanco'), label='Tipo de Vino')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# (A COMPLETAR) Comenta aquí tus observaciones sobre la separación con UMAP
# "UMAP parece ofrecer [una excelente/buena/regular] visualización, con clusters [más compactos/más dispersos] y [mejor/peor] preservación de la estructura global..."
# "La separación de los tipos de vino original con UMAP es..."


<a name="conclusion_reduccion"></a>
### 5.5. (A COMPLETAR) Tareas Finales de Reducción y Conclusión
Una vez generadas todas las proyecciones visuales, es momento de analizar cuál método nos ofrece la visión más clara, perspicaz y útil de la estructura de nuestros datos y de los clusters encontrados.

#### TAREAS EN EL NOTEBOOK:
1.  **Comparación Visual Exhaustiva:**
    * Vuelvan a revisar todos los gráficos de PCA, t-SNE y UMAP (con clusters de KMeans, DBSCAN y el tipo de vino original).
    * ¿Cuál proyección muestra la mejor separación entre los clusters encontrados por KMeans y DBSCAN?
    * ¿Cuál preserva mejor la separación natural entre vinos tintos y blancos?
    * ¿Observan alguna estructura interesante (sub-grupos, outliers, continuidad) en alguna de las visualizaciones que no era evidente antes?

```
(A COMPLETAR) Compara aquí las visualizaciones y la separación de clusters/tipos de vino:

"Al comparar las proyecciones de PCA, t-SNE y UMAP, se observa que [t-SNE/UMAP/PCA] es el que mejor logra visualizar la separación de los clusters de [KMeans/DBSCAN], mientras que [otro método] es mejor para el tipo de vino original porque..."

"Algunas estructuras interesantes observadas son..."
```

2.  **Consideraciones de Rendimiento:**
    * Comenten brevemente sobre el tiempo de ejecución aproximado de cada método de reducción de dimensionalidad (PCA es rápido, t-SNE es lento, UMAP intermedio).

```
(A COMPLETAR) Comentarios sobre el rendimiento:

"En términos de velocidad, PCA fue el más rápido, seguido por UMAP. t-SNE fue el más lento, lo cual es de esperar para datasets de este tamaño."
```

3.  **Justificar:**
    * Escriban una conclusión clara: ¿Qué técnica de reducción de dimensionalidad (PCA, t-SNE o UMAP) consideran más efectiva para **visualizar** la estructura de este dataset de vinos en particular, y por qué?
    * Argumenten su elección basándose en la claridad de los clusters, la preservación de la estructura original y la eficiencia computacional.

```
(A COMPLETAR) Mi Conclusión sobre la Mejor Técnica de Reducción de Dimensionalidad para Visualización:

Basado en [la claridad visual, la preservación de estructuras y el rendimiento], considero que [PCA / t-SNE / UMAP] es la técnica más efectiva para visualizar este dataset porque...

```

<a name="final"></a>
# 6. Conclusiones: A COMPLETAR Conclusiones Finales del Proyecto
En esta sección final, resume tus hallazgos clave de todo el proyecto integrador del Módulo 3. ¡Es tu oportunidad para mostrar todo lo que aprendiste!

#### TAREAS EN EL NOTEBOOK:

1.  **Resumen de Hallazgos Clave:**
    * ¿Qué descubriste sobre la estructura intrínseca de los datos de vino a través del clustering?
    * ¿Cómo te ayudaron las técnicas de reducción de dimensionalidad a entender mejor estas estructuras? ¿Hubo alguna visualización que te sorprendiera o te diera una nueva perspectiva?
    * ¿Consideras que los clusters encontrados se correlacionan con características significativas del vino (ej. tipo, calidad, otros perfiles no obvios)?

2.  **Desafíos y Aprendizajes:**
    * ¿Cuáles fueron los principales desafíos al trabajar con modelos no supervisados y reducción de dimensionalidad en este dataset?
    * ¿Qué nuevos aprendizajes clave te llevas de este módulo y este proyecto?

3.  **Aplicaciones Futuras:**
    * ¿Cómo crees que estas técnicas no supervisadas podrían aplicarse en escenarios reales de la industria del vino o en otros dominios? (ej. segmentación de clientes, detección de anomalías, descubrimiento de nuevas categorías de productos).


```
(A COMPLETAR) Tus Conclusiones Finales Aquí:

"Este proyecto integrador nos permitió adentrarnos en el mundo del aprendizaje no supervisado, revelando patrones y estructuras ocultas en el dataset de vinos que no serían evidentes con un análisis supervisado tradicional..."

```